## Importation des librairies

In [76]:

import pandas as pd
import re

import nltk
import pandas as pd
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/camille/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Import data

In [77]:
data = pd.read_csv('/Users/camille/repo/Hetic/projet_demo/ia_theme/scraping_projet_lois/results/output_file_cleaned.csv')
data['texts'] = data['texts'].astype(str)

## Data cleaning

In [78]:
import re
import pandas as pd
stop_words = set(stopwords.words('french'))
def clean_text(text):
    # Convertir en minuscules
    text = text.apply(lambda x: x.lower())
    
    # Enlever les apostrophes
    text = text.apply(lambda x: re.sub(r'\'', ' ', x))
    
    # Enlever la ponctuation
    text = text.apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
    
    # Enlever les stopwords
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    # Enlever les nombres
    text = text.apply(lambda x: re.sub(r'\d+', '', x))
    # Enlever les mots de moins de 3 lettres
    text = text.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))


    return text

data.rename(columns = {'texts':'motifs'}, inplace = True)
data["motifs"] = data["motifs"].astype(str)

data.rename(columns = {'helo':'projet_lois'}, inplace = True)
data["projet_lois"] = data["projet_lois"].astype(str)

data["clean_projet_lois"] = clean_text(data['projet_lois'])
data["clean_motifs"] = clean_text(data['motifs'])
data["clean_motifs"] = data["clean_motifs"].astype(str)
data["clean_projet_lois"] = data["clean_projet_lois"].astype(str)


## Analyse globales 
* ne prend pas en comptes les dates de publication

### Ètapes 
* clean_projet_lois	 est mis en dans liste clean_projet_lois_list
* Vectorisation des mots rapide

In [5]:
from collections import Counter

!pip install spacy-lookups-data fr_core_news_sm
import spacy


You should consider upgrading via the '/Users/camille/repo/Hetic/projet_demo/ia_theme/ai_theme_venv/bin/python3 -m pip install --upgrade pip' command.


#### Tratements des mots
*  toutes les colonnes sont transformées en liste `clean_projet_lois_list`
*  Lemmatisation des mots
*  Comptage des mots

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [82]:
text = " ".join(data["clean_motifs"])
text = text.split()
text = [word for word in text if word not in stop_words]
text = [word for word in text if len(word) > 2] 
text = Counter(text)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
vectorizer.get_feature_names_out()

In [84]:
len(text)

12901804

In [96]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

text = " ".join(data["clean_motifs"])
text = text.split()
text = [word for word in text if word not in stop_words]
text = [word for word in text if len(word) > 2] 
text_counter = Counter(text)

# Display word counts
print(text_counter)

vectorizer = CountVectorizer()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
feature_names = vectorizer.get_feature_names_out()
feature_counts = X.sum(axis=0)

# Convert feature_names and feature_counts to DataFrame and save to CSV
df = pd.DataFrame({'feature_names': feature_names, 'count': feature_counts.tolist()[0]})
df.to_csv('feature_names.csv', index=False)


Counter({'ainsi': 20777, 'alinéa': 17438, 'mots': 14110, 'loi': 9449, 'présent': 8782, 'après': 8486, 'état': 7404, 'peut': 6959, 'conditions': 6839, 'cette': 6556, 'modifié': 6388, 'rédigé': 6222, 'application': 6108, 'conseil': 5963, 'être': 5953, 'iii': 5766, 'dispositions': 5699, 'sécurité': 5496, 'cas': 5466, 'sociale': 5093, 'articles': 5061, 'santé': 5050, 'art': 5049, 'phrase': 4501, 'prévues': 4395, 'remplacés': 4395, 'décret': 4289, 'plus': 3911, 'modalités': 3872, 'mentionnés': 3839, 'dont': 3766, 'public': 3749, 'personnes': 3684, 'deuxième': 3652, 'mentionnées': 3577, 'lorsque': 3455, 'publique': 3435, 'délai': 3374, 'entre': 3257, 'compter': 3237, 'non': 3224, 'mise': 3176, 'ordonnance': 3168, 'autorité': 3167, 'droit': 3161, 'nouveau': 3158, 'première': 3141, 'leurs': 3133, 'peuvent': 3132, 'mois': 3130, 'sous': 3098, 'bis': 3080, 'personne': 3050, 'chapitre': 3027, 'travail': 2992, 'deux': 2990, 'services': 2952, 'dernier': 2950, 'notamment': 2945, 'durée': 2918, 'menti

In [97]:
count_word = pd.read_csv('feature_names.csv')

In [109]:
count_word_top100.to_csv('count_word_top100.csv', index=False)

In [85]:
import pandas as pd
import spacy
import nltk 
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the French model
nlp = spacy.load("fr_core_news_sm")
data['projet_lois'].astype(str)
# Input text
text = data['projet_lois']

# Split the text into chunks of up to 1,000,000 characters
chunk_size = 1000000
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

tfidf = TfidfVectorizer(tokenizer=lambda text: [token.text for token in nlp(text)],
                        lowercase=True)
X = tfidf.fit_transform(data["clean_projet_lois"])
feature_names = tfidf.get_feature_names_out()


for i, doc in enumerate(data['clean_projet_lois']):
    print("Document {}:".format(i))
    indices = X[i].indices
    scores = X[i].data
    for j in range(len(indices)):
        print("{:<15} {:.4f}".format(feature_names[indices[j]], scores[j]))
df_tfidf = pd.DataFrame(X.toarray(), columns=tfidf.get_feature_names_out())

selected_cols = ['date', 'clean_projet_lois']
data_selected = data[selected_cols]

df_tfidf_test = pd.concat([data_selected, df_tfidf], axis=1)



/Users/camille/repo/Hetic/projet_demo/ia_theme/ai_theme_venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Document 0:
maire           0.0100
bruno           0.0106
philippe        0.0060
édouard         0.0060
signé           0.0092
juin            0.0082
paris           0.0046
fait            0.0046
protection      0.0195
assurantiel     0.0350
groupe          0.0260
société         0.0231
institution     0.0614
prévoyance      0.0585
institutions    0.1010
leurs           0.0089
sociale         0.0277
sécurité        0.0085
mutuelles       0.0666
unions          0.2404
mutuelle        0.1001
occurrences     0.0126
deux            0.0092
chacune         0.0112
union           0.2261
mutualité       0.0143
ier             0.0096
complété        0.0094
troisième       0.0184
adhérant        0.0160
établissements  0.0098
défaillante     0.0175
lequel          0.0123
laquelle        0.0104
huitième        0.0239
rétribution     0.0175
recevoir        0.0118
rédigée         0.0199
sixième         0.0212
fin             0.0186
cinquième       0.0210
soumises        0.0123
remplacée       0.0096

In [12]:

selected_cols = ['date', 'clean_projet_lois']
data_selected = data[selected_cols]

df_tfidf_test = pd.concat([data_selected, df_tfidf], axis=1)


In [13]:
df_tfidf_test

,date,clean_projet_lois,aap,abaissement,abaisser,abaissé,abaissés,abandon,abandonne,abandonner,...,événement,événements,être,êtres,île,îles,îlots,œuvrant,œuvre,œuvres
0,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,29 juin 2017.,projet loi premier ministre rapport ministre t...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.019558,0.0,0.0,0.0,0.0,0.0,0.031824,0.0
2,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,22 février 2022.,nan,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
444,23 février 2022.,projet loi premier ministre rapport ministre t...,0.0,0.0,0.0,0.02321,0.0,0.0,0.0,0.0,...,0.0,0.0,0.050052,0.0,0.0,0.0,0.0,0.0,0.013574,0.0
445,7 mars 2022.,projet loi premier ministre rapport ministre é...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
446,9 mars 2022.,projet loi premier ministre rapport ministre t...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [14]:
df_tfidf_test.to_csv('/Users/camille/repo/Hetic/projet_demo/ia_theme/scraping_projet_lois/results/output_file_tfidf.csv')

In [14]:
df_tfidf_test = pd.read_csv('/Users/camille/repo/Hetic/projet_demo/ia_theme/scraping_projet_lois/results/output_file_tfidf.csv')

In [16]:
df_tfidf_test.dropna(subset=['clean_projet_lois'], inplace=True)

In [17]:
df_tfidf_test

,Unnamed: 0,date,clean_projet_lois,aap,abaissement,abaisser,abaissé,abaissés,abandon,abandonne,...,événement,événements,être,êtres,île,îles,îlots,œuvrant,œuvre,œuvres
0,0,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,1,29 juin 2017.,projet loi premier ministre rapport ministre t...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.019558,0.0,0.0,0.0,0.0,0.0,0.031824,0.0
2,2,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,3,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,4,29 juin 2017.,projet loi premier ministre rapport ministre s...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,440,2 février 2022.,projet loi premier ministre rapport ministre t...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
444,444,23 février 2022.,projet loi premier ministre rapport ministre t...,0.0,0.0,0.0,0.02321,0.0,0.0,0.0,...,0.0,0.0,0.050052,0.0,0.0,0.0,0.0,0.0,0.013574,0.0
445,445,7 mars 2022.,projet loi premier ministre rapport ministre é...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
446,446,9 mars 2022.,projet loi premier ministre rapport ministre t...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


#### Wordcloud

In [50]:
from wordcloud import WordCloud

In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Calculate the frequency count for each word
word_freq = df_tfidf_test.iloc[:, 2:].sum().sort_values(ascending=False)

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# Display the word cloud
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


NameError: name 'df_tfidf_test' is not defined

In [73]:
word_freq

NameError: name 'word_freq' is not defined

### N-grams

In [21]:
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
data = df_tfidf_test.clean_projet_lois.values.tolist()  


In [34]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['projet', 'loi', 'premier', 'ministre', 'rapport', 'ministre', 'solidarites', 'sante', 'article', 'constitution', 'decrete', 'present', 'projet', 'loi', 'ratifiant', 'ordonnance', 'avril', 'relative', 'creation', 'organismes', 'dedies', 'exercice', 'activite', 'retraite', 'professionnelle', 'supplementaire', 'adaptation', 'regimes', 'retraite', 'supplementaire', 'unites', 'rente', 'delibere', 'conseil', 'ministres', 'apres', 'avis', 'conseil', 'etat', 'presente', 'assemblee', 'nationale', 'ministre', 'economie', 'finances', 'charge', 'exposer', 'motifs', 'soutenir', 'discussion', 'article', 'ordonnance', 'avril', 'relative', 'creation', 'organismes', 'dedies', 'exercice', 'activite', 'retraite', 'professionnelle', 'supplementaire', 'adaptation', 'regimes', 'retraite', 'supplementaire', 'unites', 'rente', 'ratifiee', 'article', 'chapitre', 'iii', 'titre', 'livre', 'code', 'assurances', 'ainsi', 'modifie', 'article', 'ainsi', 'modifie', 'apres', 'mot', 'entreprise', 'inseres', 'mots', 

In [35]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['projet', 'loi', 'premier', 'ministre', 'rapport', 'ministre', 'solidarites', 'sante', 'article', 'constitution_decrete', 'present', 'projet', 'loi', 'ratifiant_ordonnance_avril', 'relative', 'creation', 'organismes', 'dedies', 'exercice', 'activite', 'retraite_professionnelle_supplementaire', 'adaptation', 'regimes', 'retraite', 'supplementaire', 'unites', 'rente', 'delibere', 'conseil', 'ministres', 'apres', 'avis', 'conseil', 'etat_presente_assemblee', 'nationale', 'ministre', 'economie_finances', 'charge_exposer_motifs', 'soutenir_discussion', 'article', 'ordonnance', 'avril', 'relative', 'creation', 'organismes', 'dedies', 'exercice', 'activite', 'retraite_professionnelle_supplementaire', 'adaptation', 'regimes', 'retraite', 'supplementaire', 'unites', 'rente', 'ratifiee', 'article', 'chapitre', 'iii', 'titre', 'livre', 'code', 'assurances', 'ainsi', 'modifie', 'article', 'ainsi', 'modifie', 'apres', 'mot', 'entreprise', 'inseres', 'mots', 'fonds', 'retraite_professionnelle_suppl

In [65]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [69]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
!python -m spacy download fr_core_news_sm

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('fr_core_news_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

     |████████████████████████████████| 16.3 MB 6.3 MB/s eta 0:00:01
You should consider upgrading via the '/Users/camille/repo/Hetic/projet_demo/ia_theme/ai_theme_venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
[['projet', 'rapport', 'solidarite', 'ser', 'constitution_decrete', 'preser', 'projet', 'ratifiant_ordonnance', 'avril', 'relatif', 'creation', 'organisme', 'dedie', 'exercice', 'activite', 'professionnelle_supplementaire', 'adaptation', 'retraite', 'supplementair', 'unite', 'rent', 'deliber', 'ministre', 'apre', 'avi', 'etat', 'presente_assemblee', 'national', 'economie_finance', 'charge', 'exposer_motif', 'soutenir_discussion', 'ordonnance', 'avril', 'relatif', 'creation', 'organisme', 'dedie', 'exercice', 'activite', 'professionnelle_supplementaire', 'adaptation', 'retraite', 'supplementair', 'unite', 'rente', 'ratifiee', 'chapitrer', 'livre', 'assurance', 'modifier',

In [37]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 2), (1, 2), (2, 5), (3, 3), (4, 1), (5, 15), (6, 25), (7, 10), (8, 26), (9, 4), (10, 2), (11, 1), (12, 2), (13, 1), (14, 1), (15, 2), (16, 5), (17, 1), (18, 2), (19, 2), (20, 1), (21, 2), (22, 1), (23, 2), (24, 2), (25, 1), (26, 3), (27, 1), (28, 5), (29, 1), (30, 5), (31, 2), (32, 1), (33, 9), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 2), (40, 12), (41, 2), (42, 2), (43, 1), (44, 1), (45, 7), (46, 5), (47, 3), (48, 1), (49, 3), (50, 2), (51, 1), (52, 1), (53, 1), (54, 5), (55, 13), (56, 62), (57, 1), (58, 1), (59, 4), (60, 1), (61, 1), (62, 4), (63, 1), (64, 1), (65, 3), (66, 10), (67, 2), (68, 7), (69, 1), (70, 3), (71, 1), (72, 31), (73, 2), (74, 2), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 2), (82, 16), (83, 1), (84, 6), (85, 2), (86, 14), (87, 17), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 3), (95, 2), (96, 1), (97, 1), (98, 1), (99, 2), (100, 2), (101, 1), (102, 25), (103, 1), (104, 5), (105, 2)]]


In [45]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[[(0, 2), (1, 2), (2, 5), (3, 3), (4, 1), (5, 15), (6, 25), (7, 10), (8, 26), (9, 4), (10, 2), (11, 1), (12, 2), (13, 1), (14, 1), (15, 2), (16, 5), (17, 1), (18, 2), (19, 2), (20, 1), (21, 2), (22, 1), (23, 2), (24, 2), (25, 1), (26, 3), (27, 1), (28, 5), (29, 1), (30, 5), (31, 2), (32, 1), (33, 9), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 2), (40, 12), (41, 2), (42, 2), (43, 1), (44, 1), (45, 7), (46, 5), (47, 3), (48, 1), (49, 3), (50, 2), (51, 1), (52, 1), (53, 1), (54, 5), (55, 13), (56, 62), (57, 1), (58, 1), (59, 4), (60, 1), (61, 1), (62, 4), (63, 1), (64, 1), (65, 3), (66, 10), (67, 2), (68, 7), (69, 1), (70, 3), (71, 1), (72, 31), (73, 2), (74, 2), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 2), (82, 16), (83, 1), (84, 6), (85, 2), (86, 14), (87, 17), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 3), (95, 2), (96, 1), (97, 1), (98, 1), (99, 2), (100, 2), (101, 1), (102, 25), (103, 1), (104, 5), (105, 2)]]
[(0,
  '0.000*"article" + 0.000*"m

In [42]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"justice" + 0.013*"plus" + 0.009*"ameliorer" + 0.009*"mieux" + '
  '0.008*"mission" + 0.008*"reforme" + 0.007*"efficacite" + 0.006*"charge" + '
  '0.006*"personnel" + 0.006*"peine"'),
 (1,
  '0.093*"ministre" + 0.037*"loi" + 0.034*"conseil" + 0.033*"premier" + '
  '0.031*"projet" + 0.031*"article" + 0.020*"relatif" + 0.019*"etat" + '
  '0.018*"national" + 0.016*"rapport"'),
 (2,
  '0.080*"mot" + 0.032*"article" + 0.031*"retraite" + 0.021*"regime" + '
  '0.019*"complementaire_obligatoire" + 0.018*"apre" + 0.017*"remplace" + '
  '0.017*"alinea" + 0.016*"insere" + 0.014*"base"'),
 (3,
  '0.004*"article" + 0.002*"mot" + 0.001*"code" + 0.001*"ainsi" + '
  '0.001*"alinea" + 0.001*"pouvoir" + 0.001*"present" + 0.001*"disposition" + '
  '0.001*"condition" + 0.001*"social"'),
 (4,
  '0.006*"article" + 0.003*"code" + 0.003*"mot" + 0.002*"alinea" + '
  '0.002*"ainsi" + 0.001*"disposition" + 0.001*"titre" + 0.001*"mentionnee" + '
  '0.001*"pouvoir" + 0.001*"apre"'),
 (5,
  '0.032*"ar

In [43]:
print ('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.0622094668107565


In [56]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.413820773535384

Coherence Score:  0.39092459799213963


In [70]:
stop_words = ["article", "mot", "alinea", "code", "disposition", "lois", "titre", "ministre", "premier", "loi", 'conseil', 'gouvernement', 'ainsi', 'denormandie', "presente_assemblee", "relatif", "philipp"]
id2word = corpora.Dictionary([doc for doc in data_lemmatized if doc not in stop_words])
texts = [[word for word in doc if word not in stop_words] for doc in data_lemmatized if doc not in stop_words]
corpus = [id2word.doc2bow(text) for text in texts]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [71]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"retraite" + 0.012*"social" + 0.009*"condition" + 0.009*"prevue" + '
  '0.008*"apre" + 0.007*"securit" + 0.007*"present" + 0.007*"pouvoir" + '
  '0.006*"mentionne" + 0.006*"regime"'),
 (1,
  '0.013*"social" + 0.010*"pouvoir" + 0.009*"public" + 0.009*"condition" + '
  '0.008*"apre" + 0.008*"sante" + 0.008*"present" + 0.007*"mentionnee" + '
  '0.007*"securit" + 0.007*"etablissement"'),
 (2,
  '0.014*"projet" + 0.013*"etat" + 0.010*"apre" + 0.010*"formation" + '
  '0.010*"national" + 0.009*"ordonnance" + 0.008*"public" + 0.008*"condition" '
  '+ 0.008*"travail" + 0.007*"professionnel"'),
 (3,
  '0.025*"recherche" + 0.009*"plus" + 0.007*"scientifique" + 0.007*"public" + '
  '0.007*"innovation" + 0.006*"etablissement" + 0.006*"permettre" + '
  '0.005*"projet" + 0.005*"notamment" + 0.005*"etat"'),
 (4,
  '0.014*"pouvoir" + 0.010*"apre" + 0.009*"public" + 0.008*"etat" + '
  '0.008*"condition" + 0.008*"prevue" + 0.008*"application" + 0.007*"redige" + '
  '0.007*"remplace" + 0.007

In [72]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/camille/repo/Hetic/projet_demo/ia_theme/ai_theme_venv/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.061228  0.002435       1        1  33.089033
1     -0.040244  0.031855       2        1  26.350288
2     -0.055957 -0.102078       3        1  18.301930
0     -0.056871  0.069358       4        1  14.182198
3      0.214300 -0.001570       5        1   8.076552, topic_info=          Term        Freq        Total Category  logprob  loglift
913  recherche  692.000000   692.000000  Default  30.0000  30.0000
82    retraite  472.000000   472.000000  Default  29.0000  29.0000
67      projet  935.000000   935.000000  Default  28.0000  28.0000
292  formation  478.000000   478.000000  Default  27.0000  27.0000
878       plus  450.000000   450.000000  Default  26.0000  26.0000
..         ...         ...          ...      ...      ...      ...
526       tout   48.789534   409.009962   Topic5  -5.8156   0.3900
859   objectif   44.778701   308.606172   Topic5  -5.9014   0.5859
228     devoir   45.942764   404.497561   Topic5  -5.8757   0.3410
425    pouvoir   52.609076  1979.435411   Topic5  -5.7403  -1.1115
139      autre   47.149734   577.427003   Topic5  -5.8498   0.0110

[443 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1608      1  0.115115       abaisse
1608      4  0.863361       abaisse
6214      2  0.945614      academie
9108      2  0.950516  acces_direct
6997      2  0.963285  acces_precoc
...     ...       ...           ...
2743      4  0.016507        visite
2286      1  0.915277          vote
2286      2  0.015255          vote
2286      3  0.045764          vote
2286      4  0.030509          vote

[1052 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 1, 4])

In [1]:
from typing import Callable

def multiply_numbers(x: int, y: int) -> int:
    return x * y

def foo(x: int, y: int, func: Callable) -> int:
    output = func(x, y)
    return output

foo(2, 3, multiply_numbers)  # Error: multiply_numbers returns a different type than expected


6

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_files

reviews_train = load_files("aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target

print("Number of documents in train data: {}".format(len(text_train)))
print("Samples per class (train): {}".format(np.bincount(y_train)))

reviews_test = load_files("aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target

print("Number of documents in test data: {}".format(len(text_test)))
print("Samples per class (test): {}".format(np.bincount(y_test)))

vect = CountVectorizer(min_df=5, ngram_range=(2, 2))
X_train = vect.fit_transform(text_train)
X_test = vect.transform(text_test)

print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("X_train:\n{}".format(repr(X_train)))
print("X_test: \n{}".format(repr(X_test)))

feature_names = vect.get_feature_names_out()
print("Number of features: {}".format(len(feature_names)))

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)

lr = grid.best_estimator_
y_pred = lr.predict(X_test)
print("Score: {:.2f}".format(lr.score(X_test, y_test)))

Number of documents in train data: 75000
Samples per class (train): [12500 12500 50000]
Number of documents in test data: 25000
Samples per class (test): [12500 12500]
Vocabulary size: 332885
X_train:
<75000x332885 sparse matrix of type '<class 'numpy.int64'>'
	with 12160550 stored elements in Compressed Sparse Row format>
X_test: 
<25000x332885 sparse matrix of type '<class 'numpy.int64'>'
	with 3852008 stored elements in Compressed Sparse Row format>
Number of features: 332885


/Users/camille/repo/Hetic/projet_demo/ia_theme/ai_theme_venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/camille/repo/Hetic/projet_demo/ia_theme/ai_theme_venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver option

Best cross-validation score: 0.67
Best parameters:  {'C': 0.01}
Best estimator:  LogisticRegression(C=0.01)
Score: 0.06


/Users/camille/repo/Hetic/projet_demo/ia_theme/ai_theme_venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [2]:
import nltk
from nltk.tokenize import word_tokenize

# Télécharger les données nécessaires pour le traitement du langage naturel en français
nltk.download('punkt')
nltk.download('stopwords')

# Définir la chaîne de texte en français
texte = "Le chat dort sur le tapis. Le chien court dans le jardin. Les oiseaux chantent dans les arbres."

# Tokenizer la chaîne de texte en mots individuels
mots = word_tokenize(texte, language='french')

# Supprimer les mots vides (stopwords) de la liste de mots
mots_sans_stopwords = [mot for mot in mots if mot.lower() not in nltk.corpus.stopwords.words('french')]

# Créer le sac de mots
sac_de_mots = {}
for mot in mots_sans_stopwords:
    if mot in sac_de_mots:
        sac_de_mots[mot] += 1
    else:
        sac_de_mots[mot] = 1

# Afficher le sac de mots
print(sac_de_mots)


{'chat': 1, 'dort': 1, 'tapis': 1, '.': 3, 'chien': 1, 'court': 1, 'jardin': 1, 'oiseaux': 1, 'chantent': 1, 'arbres': 1}


[nltk_data] Downloading package punkt to /Users/camille/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/camille/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
